In [1]:
import numpy as np
import keras
import pretty_midi
import librosa

import os
import sys
import mir_eval
import glob

import numpy as np

sys.path.append("..")
sys.path.append("../src")

%load_ext autoreload
%autoreload 
from src import preprocess
from src import constants
from src import datagen
from src.infer import infer_from_processed
from src.metrics import pm_to_pitches_intervals, get_f1_score_notes, get_f1_score_frames
from src import helpers


Using TensorFlow backend.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dirname = '/floyd/input/new_maestro/'
preprocess.preprocess(dirname, False)

32
saved /floyd/home/test_data_proc_scale_mm/x_0.npy
saved /floyd/home/test_data_proc_scale_mm/y_0.npy
saved /floyd/home/test_data_proc_scale_mm/x_1.npy
saved /floyd/home/test_data_proc_scale_mm/y_1.npy
saved /floyd/home/test_data_proc_scale_mm/x_2.npy
saved /floyd/home/test_data_proc_scale_mm/y_2.npy
saved /floyd/home/test_data_proc_scale_mm/x_3.npy
saved /floyd/home/test_data_proc_scale_mm/y_3.npy
saved /floyd/home/test_data_proc_scale_mm/x_4.npy
saved /floyd/home/test_data_proc_scale_mm/y_4.npy
saved /floyd/home/test_data_proc_scale_mm/x_5.npy
saved /floyd/home/test_data_proc_scale_mm/y_5.npy
saved /floyd/home/test_data_proc_scale_mm/x_6.npy
saved /floyd/home/test_data_proc_scale_mm/y_6.npy
saved /floyd/home/test_data_proc_scale_mm/x_7.npy
saved /floyd/home/test_data_proc_scale_mm/y_7.npy
saved /floyd/home/test_data_proc_scale_mm/x_8.npy
saved /floyd/home/test_data_proc_scale_mm/y_8.npy
saved /floyd/home/test_data_proc_scale_mm/x_9.npy
saved /floyd/home/test_data_proc_scale_mm/y_9.n

In [2]:
model = '/floyd/home/final_model.h5'
test_generator = datagen.DataGenerator(constants.TEST_PROCESSED_DIR, constants.SEQUENCE_SIZE, constants.BATCH_SIZE, constants.CONTEXT_WINDOW_SIZE, 'x_[0-9]+.npy', 'y_[0-9]+.npy')

note_f1s = []
note_precisions = []
note_recalls = []

frame_f1s = []
frame_precisions = []
frame_recalls = []

for sample in glob.glob(os.path.join(constants.TEST_PROCESSED_DIR, 'x_*')):
    isolated_filename = sample.split('/')[-1]
    
    gt_file= os.path.join(constants.TEST_PROCESSED_DIR,test_generator.corresponding_y(isolated_filename))
    referances = np.load(gt_file, mmap_mode='r')
    predictions = infer_from_processed(model, sample)
    
    padded =  np.zeros((predictions.shape[0], 128))
    padded[:, 21:109] = predictions
    padded =  padded.T
    sequence_est = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0).instruments[0]
    padded =  np.zeros((referances.shape[0], 128))
    padded[:, 21:109] = referances
    padded =  padded.T
    sequence_ref = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0).instruments[0]
    
    est_intervals,est_pitches = pm_to_pitches_intervals(sequence_est)
    ref_intervals,ref_pitches = pm_to_pitches_intervals(sequence_ref)
    
    note_precision, note_recall, note_f1 = get_f1_score_notes(ref_intervals, ref_pitches, est_intervals, est_pitches)
    
    frame_precision, frame_recall, frame_f1 = get_f1_score_frames(referances, predictions)
    
    
    note_f1s.append(note_f1)
    note_precisions.append(note_precision)
    note_recalls.append(note_recall)
    
    frame_f1s.append(frame_f1)
    frame_precisions.append(frame_precision)
    frame_recalls.append(frame_recall)

    
# we calculate mean across all scores as done in Ondets and Frames paper for consistency
avg_note_f1 = np.mean(note_f1s)
avg_note_precision = np.mean(note_precisions)
avg_note_recall = np.mean(note_recalls)

avg_frame_f1 = np.mean(frame_f1s)
avg_frame_precision = np.mean(frame_precisions)
avg_frame_recall = np.mean(frame_recalls)

print('Frame Score:')
print([avg_frame_precision, avg_frame_recall, avg_frame_f1])
print('Note Score:')
print([avg_note_precision, avg_note_recall, avg_note_f1])

['x_23.npy', 'x_26.npy', 'x_3.npy', 'x_17.npy', 'x_30.npy', 'x_31.npy', 'x_28.npy', 'x_21.npy', 'x_24.npy', 'x_1.npy', 'x_19.npy', 'x_12.npy', 'x_7.npy', 'x_18.npy', 'x_22.npy', 'x_13.npy', 'x_6.npy', 'x_4.npy', 'x_15.npy', 'x_25.npy', 'x_5.npy', 'x_0.npy', 'x_27.npy', 'x_11.npy', 'x_2.npy', 'x_16.npy', 'x_9.npy', 'x_8.npy', 'x_29.npy', 'x_20.npy', 'x_10.npy', 'x_14.npy']
['y_9.npy', 'y_30.npy', 'y_12.npy', 'y_5.npy', 'y_8.npy', 'y_17.npy', 'y_24.npy', 'y_16.npy', 'y_29.npy', 'y_26.npy', 'y_27.npy', 'y_20.npy', 'y_14.npy', 'y_0.npy', 'y_18.npy', 'y_13.npy', 'y_4.npy', 'y_21.npy', 'y_3.npy', 'y_23.npy', 'y_15.npy', 'y_2.npy', 'y_11.npy', 'y_25.npy', 'y_7.npy', 'y_22.npy', 'y_31.npy', 'y_1.npy', 'y_19.npy', 'y_6.npy', 'y_28.npy', 'y_10.npy']
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
one up
one up
o

In [3]:
import pandas as pd

eval_df = pd.DataFrame({'Precison' : [avg_frame_precision,avg_note_precision], 
                   'Recall' : [avg_frame_recall,avg_note_recall],
                   'F1' : [avg_frame_f1,avg_note_f1],
                    '':['Frame', 'Note']})


eval_df.set_index('')

,Precison,Recall,F1
,,,
Frame,0.755206,0.751454,0.750039
Note,0.637897,0.478511,0.540015
